<a href="https://colab.research.google.com/github/fassir/treinamento_ia_reconhecimento_imagem/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transferência de aprendizado / ajuste fino
 
Este tutorial irá guiá-lo pelo processo de uso do _transfer learning_ para aprender um classificador de imagens preciso a partir de um número relativamente pequeno de amostras de treinamento. De modo geral, transfer learning refere-se ao processo de aproveitar o conhecimento aprendido em um modelo para o treinamento de outro modelo.

Mais especificamente, o processo envolve pegar uma rede neural existente que foi previamente treinada para um bom desempenho em um conjunto de dados maior e usá-la como base para um novo modelo que aproveita a precisão da rede anterior para uma nova tarefa. Esse método se tornou popular nos últimos anos para melhorar o desempenho de uma rede neural treinada em um conjunto de dados pequeno; a intuição é que o novo conjunto de dados pode ser pequeno demais para treinar até um bom desempenho sozinho, mas sabemos que a maioria das redes neurais treinadas para aprender características de imagens geralmente aprende características semelhantes de qualquer maneira, especialmente nas camadas iniciais, onde são mais genéricas (detectores de borda, blobs, etc.).
 
O transfer learning foi amplamente viabilizado pelo código aberto de modelos de ponta; para os modelos de melhor desempenho em tarefas de classificação de imagens (como do [ILSVRC](http://www.image-net.org/challenges/LSVRC/)), é prática comum agora não apenas publicar a arquitetura, mas também liberar os pesos treinados do modelo. Isso permite que amadores usem esses classificadores de imagem de ponta para impulsionar o desempenho de seus próprios modelos específicos de tarefa.

#### Extração de características vs. ajuste fino
 
Em um extremo, transfer learning pode envolver pegar a rede pré-treinada e congelar os pesos, usando uma de suas camadas ocultas (geralmente a última) como extrator de características, usando essas características como entrada para uma rede neural menor.

No outro extremo, começamos com a rede pré-treinada, mas permitimos que alguns dos pesos (geralmente a última camada ou as últimas camadas) sejam modificados. Outro nome para esse procedimento é "ajuste fino" porque estamos ajustando levemente os pesos da rede pré-treinada para a nova tarefa. Normalmente treinamos tal rede com uma taxa de aprendizado menor, já que esperamos que as características já sejam relativamente boas e não precisem ser muito alteradas.

Às vezes, fazemos algo intermediário: congelamos apenas as camadas iniciais/genéricas, mas fazemos ajuste fino nas camadas finais. Qual estratégia é melhor depende do tamanho do seu conjunto de dados, do número de classes e de quanto ele se assemelha ao conjunto de dados em que o modelo anterior foi treinado (e, portanto, se pode se beneficiar dos mesmos extratores de características aprendidos). Uma discussão mais detalhada sobre como escolher a estratégia pode ser encontrada em [[1]](http://cs231n.github.io/transfer-learning/) [[2]](http://sebastianruder.com/transfer-learning/).
 
## Procedimento
 
Neste guia, vamos passar pelo processo de carregar um classificador de imagens de ponta com 1000 classes, o [VGG16](https://arxiv.org/pdf/1409.1556.pdf), que [venceu o desafio ImageNet em 2014](http://www.robots.ox.ac.uk/~vgg/research/very_deep/), e usá-lo como extrator de características fixo para treinar um classificador personalizado menor em nossas próprias imagens, embora com poucas alterações no código você também possa tentar o ajuste fino.

Primeiro, vamos carregar o VGG16 e remover sua camada final, a camada de classificação softmax de 1000 classes específica do ImageNet, e substituí-la por uma nova camada de classificação para as classes que estamos treinando. Em seguida, vamos congelar todos os pesos da rede, exceto os novos que conectam à nova camada de classificação, e então treinar a nova camada de classificação em nosso novo conjunto de dados.

Também vamos comparar esse método com o treinamento de uma pequena rede neural do zero no novo conjunto de dados e, como veremos, isso irá melhorar drasticamente nossa precisão. Faremos essa parte primeiro.

Como nosso exemplo, usaremos o conjunto de dados **Microsoft Cats and Dogs** com cerca de 25.000 imagens pertencentes a **2 categorias**: Cat e Dog. Treinaremos um classificador de imagens para distinguir entre essas duas classes. Vale notar que essa estratégia escala bem para conjuntos de imagens onde você pode ter até algumas centenas ou menos de imagens. O desempenho será menor com um número pequeno de amostras (dependendo das classes), como de costume, mas ainda assim impressionante considerando as restrições usuais.


In [ ]:
%pip install tensorflow matplotlib

%matplotlib inline

import os

#se estiver usando Theano com GPU
os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

### Obtendo um conjunto de dados

In [ ]:
!echo "Baixando kagglecatsanddogs_5340.zip para o notebook"
!curl -L -o kagglecatsanddogs_5340.zip --progress-bar https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip
!unzip -q kagglecatsanddogs_5340.zip
!ls kagglecatsanddogs_5340/PetImages

In [ ]:
root = 'kagglecatsanddogs_5340/PetImages'
categories = [os.path.join(root, c) for c in os.listdir(root) if os.path.isdir(os.path.join(root, c))]
print(categories)
train_split, val_split = 0.7, 0.15

Esta função é útil para pré-processar os dados em uma imagem e vetor de entrada.

In [ ]:
# função auxiliar para carregar imagem e retornar a imagem e o vetor de entrada
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

Carregue todas as imagens da pasta raiz

In [ ]:
from PIL import UnidentifiedImageError

data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames
              in os.walk(category) for f in filenames
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        try:
            img, x = get_image(img_path)
            data.append({'x': np.array(x[0]), 'y': c})
        except (UnidentifiedImageError, OSError):
            # Arquivo ignorado: não pôde ser aberto como imagem
            continue

# contar o número de classes
num_classes = len(categories)

Aleatorize a ordem dos dados.

In [ ]:
random.shuffle(data)

criar divisão de treino / validação / teste (70%, 15%, 15%)

In [ ]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

Separe os dados dos rótulos.

In [ ]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print("Primeiros 5 itens de y_test:", y_test[:5])

Pré-processe os dados como antes, garantindo que estejam em float32 e normalizados entre 0 e 1.

In [ ]:
# normalizar os dados
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# converter rótulos para vetores one-hot
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

Vamos obter um resumo do que temos.

In [ ]:
# resumo
print("carregamento finalizado de %d imagens de %d categorias"%(len(data), num_classes))
print("divisão treino / validação / teste: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("formato dos dados de treino: ", x_train.shape)
print("formato dos rótulos de treino: ", y_train.shape)


Se tudo funcionou corretamente, você deve ter carregado as imagens do dataset **Microsoft Cats and Dogs** e as dividido em três conjuntos: `train` (treino), `val` (validação) e `test` (teste).
 
O formato dos dados de treino será (`n`, 224, 224, 3), onde `n` é o número de imagens no conjunto de treino, 224x224 é o tamanho das imagens e 3 representa os canais de cor (RGB). Os rótulos estarão no formato (`n`, 2), pois temos **2 classes**: Cat e Dog.
 
A divisão dos dados foi feita para garantir uma avaliação correta do classificador:
- `train`: usado para treinar o modelo
- `val`: usado para validação durante o treinamento (ajuste de hiperparâmetros e prevenção de overfitting)
- `test`: usado apenas ao final, para avaliar a acurácia final do modelo em dados nunca vistos
 
Vamos rapidamente visualizar algumas imagens de exemplo do nosso conjunto de dados.

In [ ]:
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(root) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
idx = [int(len(images) * random.random()) for i in range(8)]
imgs = [image.load_img(images[i], target_size=(224, 224)) for i in idx]
concat_image = np.concatenate([np.asarray(img) for img in imgs], axis=1)
plt.figure(figsize=(16,4))
plt.imshow(concat_image)

### Primeiro, treinando uma rede neural do zero

In [ ]:
# construir a rede
model = Sequential()
print("Input dimensions: ",x_train.shape[1:])

model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Criamos uma rede de tamanho médio com cerca de 1,2 milhão de pesos e vieses (os parâmetros). A maioria deles está levando para a camada totalmente conectada pré-softmax "dense_5".
 
Agora podemos treinar nosso modelo por 10 épocas com um batch size de 128. Também registraremos seu histórico para podermos plotar a perda ao longo do tempo depois.

In [ ]:
# compilar o modelo para usar função de perda entropia cruzada categórica e otimizador adadelta
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=10,
                    validation_data=(x_val, y_val))


Vamos plotar a perda de validação e a acurácia de validação ao longo do tempo.

In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.set_title("perda de validação")
ax.set_xlabel("épocas")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_acc"])
ax2.set_title("acurácia de validação")
ax2.set_xlabel("épocas")
ax2.set_ylim(0, 1)

plt.show()

Observe que a perda de validação começa a aumentar após cerca de 16 épocas, mesmo que a acurácia de validação permaneça aproximadamente entre 40% e 50%. Isso sugere que nosso modelo começa a sobreajustar por volta desse ponto, e o melhor desempenho teria sido alcançado se tivéssemos parado antes. No entanto, nossa acurácia provavelmente não teria passado de 50%, e provavelmente seria menor.
 
Também podemos obter uma avaliação final rodando nosso modelo no conjunto de teste. Fazendo isso, obtemos os seguintes resultados:

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Perda no teste:', loss)
print('Acurácia no teste:', accuracy)

## Transferência de aprendizado começando com uma rede existente

In [ ]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

Observe que o VGG16 é _muito_ maior do que a rede que construímos anteriormente. Ele contém 13 camadas convolucionais e duas camadas totalmente conectadas no final, e possui mais de 138 milhões de parâmetros, cerca de 100 vezes mais do que a rede que fizemos acima. Como nossa primeira rede, a maioria dos parâmetros está nas conexões que levam à primeira camada totalmente conectada.
 
O VGG16 foi feito para resolver o ImageNet e atinge um [erro top-5 de 8,8%](https://github.com/jcjohnson/cnn-benchmarks), o que significa que 91,2% das amostras de teste foram classificadas corretamente entre as 5 melhores previsões para cada imagem. Sua acurácia top-1 -- equivalente à métrica de acurácia que usamos (a previsão principal está correta) -- é de 73%. Isso é especialmente impressionante, já que não são apenas 97, mas 1000 classes, o que significa que chutes aleatórios nos dariam apenas 0,1% de acurácia.
 
Para usar essa rede em nossa tarefa, "removemos" a camada final de classificação, a camada softmax de 1000 neurônios no final, que corresponde ao ImageNet, e a substituímos por uma nova camada softmax para nosso conjunto de dados, que contém 97 neurônios no caso do conjunto 101_ObjectCategories.
 
Em termos de implementação, é mais fácil simplesmente criar uma cópia do VGG da camada de entrada até a penúltima camada e trabalhar com isso, em vez de modificar o objeto VGG diretamente. Então, tecnicamente, nunca "removemos" nada, apenas contornamos/ignoramos. Isso pode ser feito da seguinte forma, usando a classe `Model` do keras para inicializar um novo modelo cuja camada de entrada é a mesma do VGG, mas cuja camada de saída é nossa nova camada softmax, chamada `new_classification_layer`. Observação: embora pareça que estamos duplicando essa grande rede, internamente o Keras está apenas copiando todas as camadas por referência, então não precisamos nos preocupar com sobrecarga de memória.

In [ ]:
# criar uma referência para a camada de entrada do VGG
inp = vgg.input

# criar uma nova camada softmax com num_classes neurônios
new_classification_layer = Dense(num_classes, activation='softmax')

# conectar nossa nova camada à penúltima camada do VGG e fazer referência a ela
out = new_classification_layer(vgg.layers[-2].output)

# criar uma nova rede entre inp e out
model_new = Model(inp, out)


Vamos re-treinar essa rede, `model_new`, no novo conjunto de dados e rótulos. Mas primeiro, precisamos congelar os pesos e vieses em todas as camadas da rede, exceto a nova no final, com a expectativa de que as características aprendidas no VGG ainda sejam bastante relevantes para a nova tarefa de classificação de imagens. Não é o ideal, mas provavelmente melhor do que conseguimos treinar com nosso conjunto de dados limitado.
 
Ao definir o parâmetro `trainable` como falso em cada camada (exceto nossa nova camada de classificação), garantimos que todos os pesos e vieses dessas camadas permaneçam fixos, e treinamos apenas os pesos da camada final. Em alguns casos, pode ser interessante *não* congelar todas as camadas pré-classificação. Se seu conjunto de dados tiver amostras suficientes e não se parecer muito com o ImageNet, pode ser vantajoso fazer ajuste fino em algumas camadas do VGG junto com o novo classificador, ou até mesmo em todas. Para isso, basta alterar o código abaixo para tornar mais camadas treináveis.
 
No caso do CalTech-101, vamos apenas fazer extração de características, temendo que o ajuste fino em excesso com esse conjunto de dados cause overfitting. Mas talvez estejamos errados? Um bom exercício seria testar ambos e comparar os resultados.
 
Portanto, vamos congelar as camadas e compilar o novo modelo com exatamente o mesmo otimizador e função de perda do nosso primeiro modelo, para uma comparação justa. Em seguida, rodamos `summary` novamente para ver a arquitetura da rede.

In [ ]:
# tornar todas as camadas não treináveis congelando os pesos (exceto a última camada)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# garantir que a última camada seja treinável/não congelada
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

Olhando o resumo, vemos que a rede é idêntica ao modelo VGG que instanciamos anteriormente, exceto pela última camada, que antes era um softmax de 1000 neurônios e agora foi substituída por um novo softmax de 97 neurônios. Além disso, ainda temos cerca de 134 milhões de pesos, mas agora a grande maioria deles são "parâmetros não treináveis" porque congelamos as camadas em que estão contidos. Agora temos apenas 397.000 parâmetros treináveis, o que é apenas um quarto do número de parâmetros necessários para treinar o primeiro modelo.
 
Como antes, vamos treinar o novo modelo, usando os mesmos hiperparâmetros (tamanho do batch e número de épocas) de antes, junto com o mesmo algoritmo de otimização. Também acompanhamos seu histórico durante o processo.

In [ ]:
history2 = model_new.fit(x_train, y_train,
                         batch_size=128,
                         epochs=10,
                         validation_data=(x_val, y_val))


Nossa acurácia de validação fica próxima de 80% ao final, o que é mais de 30% de melhoria em relação à rede original treinada do zero (ou seja, erramos 20% das amostras, em vez de 50%).
 
Vale notar também que essa rede treina _um pouco mais rápido_ do que a rede original, apesar de ter mais de 100 vezes mais parâmetros! Isso ocorre porque congelar os pesos elimina a necessidade de fazer backpropagation em todas essas camadas, economizando tempo de execução.
 
Vamos plotar novamente a perda e a acurácia de validação, agora comparando o modelo original treinado do zero (em azul) e o novo modelo com transferência de aprendizado (em verde).

In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.plot(history2.history["val_loss"])
ax.set_title("perda de validação")
ax.set_xlabel("épocas")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_acc"])
ax2.plot(history2.history["val_acc"])
ax2.set_title("acurácia de validação")
ax2.set_xlabel("épocas")
ax2.set_ylim(0, 1)

plt.show()

Observe que, enquanto o modelo original começou a sobreajustar por volta da época 16, o novo modelo continuou a diminuir lentamente sua perda ao longo do tempo, e provavelmente teria melhorado um pouco mais sua acurácia com mais iterações. O novo modelo chegou a cerca de 80% de acurácia top-1 (no conjunto de validação) e continuou melhorando lentamente até 100 épocas.
 
Talvez pudéssemos ter melhorado o modelo original com mais regularização ou mais dropout, mas dificilmente alcançaríamos o ganho de >30% em acurácia.
 
Novamente, fazemos uma validação final no conjunto de teste.

In [ ]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Perda no teste:', loss)
print('Acurácia no teste:', accuracy)

Para prever uma nova imagem, basta rodar o código a seguir para obter as probabilidades de cada classe.

In [ ]:
img, x = get_image('kagglecatsanddogs_5340/PetImages/Cat/7.jpg')
probabilities = model_new.predict([x])


### Melhorando os resultados